In [1]:
%pip install langchain-community langchain-pinecone langchain-openai pinecone -q
from google.colab import userdata
import os


In [2]:
os.environ['LANGSMITH_TRACING'] = userdata.get('LANGSMITH_TRACING')
os.environ['LANGSMITH_ENDPOINT'] = userdata.get('LANGSMITH_ENDPOINT')
os.environ['LANGSMITH_API_KEY'] = userdata.get('LANGSMITH_API_KEY')
os.environ['LANGSMITH_PROJECT'] = userdata.get('LANGSMITH_PROJECT')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')


In [14]:
# import os
# from typing import List, Dict, Tuple
# from pinecone import Pinecone
# from langchain_pinecone import PineconeVectorStore
# from langchain_openai import OpenAIEmbeddings
# import openai  # LLM 호출용

# # 1. Pinecone & 인덱스 초기화
# PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
# INDEX_NAME       = "food-index"
# EMBED_MODEL      = "text-embedding-3-small"

# pc = Pinecone(api_key=PINECONE_API_KEY)
# index = pc.Index(INDEX_NAME)
# embeddings = OpenAIEmbeddings(model=EMBED_MODEL)
# vector_store = PineconeVectorStore(index=index, embedding=embeddings)

# # 2. Pinecone 검색
# def search_menu(menu_name: str, k: int = 1) -> List[Tuple]:
#     return vector_store.similarity_search_with_score(query=menu_name, k=k)

# # 3. Pinecone 결과 → 컨텍스트 변환
# def build_context(matches: List[Tuple]) -> str:
#     lines = []
#     for doc, score in matches:
#         meta = doc.metadata or {}
#         name = meta.get("RCP_NM", "알 수 없는 메뉴")
#         kcal = meta.get("INFO_ENG", "칼로리 정보 없음")
#         lines.append(f"- 메뉴명: {name}, 칼로리: {kcal} (유사도: {score:.2f})")
#     return "\n".join(lines)

# # 4. LLM에게 칼로리만 물어보기
# def ask_llm_calorie(menu_name: str) -> str:
#     prompt = (
#         f"다음 음식의 대표적인 1인분 칼로리(kcal) 숫자만 알려주세요 ** 반드시 숫자만 반환!! **:\n"
#         f"‘{menu_name}’"
#     )
#     resp = openai.chat.completions.create(
#         model="gpt-4o-mini",
#         messages=[{"role":"user","content":prompt}],
#         temperature=0
#     )
#     # LLM이 "450kcal" 또는 "450" 같은 형태로 답한다고 가정
#     return resp.choices[0].message.content.strip()

# # 5. 최종 파이프라인 (threshold = 0.7)
# def get_menu_context_with_threshold(
#     menu_name: str,
#     k: int = 1,
#     threshold: float = 0.4
# ) -> str:
#     matches = search_menu(menu_name, k)
#     # ① 결과 없거나, ② 최고 유사도 < 임계치 → LLM fallback
#     if not matches or matches[0][1] < threshold:
#         calorie = ask_llm_calorie(menu_name)
#         # 원본 포맷 그대로
#         return f"- 메뉴명: {menu_name}, 칼로리: {calorie}"
#     # 정상 검색 결과
#     return build_context(matches)

# # 6. 테스트
# if __name__ == "__main__":
#     blip_predicted_name = "해물칼국수"
#     rag_context = get_menu_context_with_threshold(blip_predicted_name)
#     print("=== 컨텍스트 ===")
#     print(rag_context)

=== 컨텍스트 ===
- 메뉴명: 해물칼국수, 칼로리: 184.6 (유사도: 0.48)


In [ ]:
# from langchain_core.prompts import PromptTemplate
# from langchain_core.output_parsers import StrOutputParser
# from langchain_openai import ChatOpenAI

# text = " 29세 , 남성, 180cm, 82kg, 오늘 점심식사고 앞서 계란볶음밥을 아침으로 먹었습니다."

# llm = ChatOpenAI(model='gpt-4.1', temperature=0.3)

# output_parser = StrOutputParser()

# prompt = PromptTemplate.from_template("""
# [System Instruction]
# -  당신은 사용자가 먹은 음식의 **이미지**와 **텍스트 정보**(성별, 나이, 키, 몸무게, 식사 정보 등)를 입력하면,
# 섭취한 열량 및 음식 정보를 분석하고, 남은 권장 열량에 맞는 식단과 운동을 추천하는 영양&운동 전문 PT 선생님인 "G-PT"입니다.

# - 음식의 이미지 정보는 Blip model을 통해 반환된 menu_name을 가지고 온다.

# - searh_menu를 통해 받아온 menu_name을 유사도 검색을 통해 유사한 음식의 정보를 추출한다. (name, kcal)

# - 우선적으로 주어진 문서내에서 정보를 추출해 답변해야 하나, 유사도가 {threshold} 보다 낮을 때는 ask_llm_calories를 통해 llm을 통해 답변한다.

# - 답변은 반드시 [Output Indicator]에 따라야 한다.

# - 절대 말을 지어내어서는 안된다!!!

# 1. **입력**
#    - 사용자는 Streamlit UI에서 음식 이미지를 첨부하고,
#      - 성별
#      - 나이
#      - 키
#      - 몸무게
#      - 오늘 식사가 하루 중 몇 번째 끼니인지(아침/점심/저녁/간식 등)
#      - 앞서 먹은 음식(있으면 목록, 없으면 없음)
#      를 텍스트로 입력합니다.

#    - **[예외처리]**
#      - 이미지가 첨부되지 않은 경우:
#        → "오늘 드신 음식의 사진이 있다면 첨부해주세요!"
#      - 성별, 나이, 키, 몸무게 중 하나라도 누락 시:
#        → "사용자의 정보가 있어야 더 정확한 열량 계산이 가능합니다. '성별', '몸무게', '키', '나이' 등의 정보 다시한번 입력해주세요!"

# 2. **음식 이미지 분류**
#    - 첨부된 음식 이미지는 BLIP 모델로 분류하여 음식 이름을 예측합니다.
#    - **[예외처리]**
#      - BLIP 분류 실패 시:
#        → "음식을 제대로 인식하지 못했습니다. 음식 사진을 다시 올려주세요"

# 3. **음식 정보 검색 (RAG Retrieval)**
#    - 분류된 음식 이름을 기반으로, Pinecone Embedding Index에서 유사 음식의 정보를 검색합니다.
#      (음식 이름, 음식 재료, 1회 섭취 기준 열량 포함)

# 4. **LLM 기반 식단/운동 종합 추천**
#    - 사용자 정보(성별/나이/키/몸무게/식사 순서/이전 식사)와
#      해당 음식 정보(이름, 재료, 열량)를 LLM에게 전달합니다.
#    - LLM은 아래를 종합 추천합니다:
#      - 오늘까지 누적 섭취 열량
#      - 남은 1일 권장 섭취 열량
#      - 남은 열량에 맞는 다음 식사(또는 하루 마무리 식단)
#      - 먹은 열량을 소모할 수 있는 운동 추천 (종목/시간/칼로리)

# ## [요약 분기]
# - 이미지 미첨부:
#   "오늘 드신 음식의 사진이 있다면 첨부해주세요!"
# - 신체 정보 미입력:
#   "사용자의 정보가 있어야 더 정확한 열량 계산이 가능합니다. '성별', '몸무게', '키', '나이' 등의 정보 다시한번 입력해주세요!"
# - 이미지 분류 실패:
#   "음식을 제대로 인식하지 못했습니다. 음식 사진을 다시 올려주세요"


# [Input Data]

# {rag_context}
# {text}

# [Output Indicator]
# "메뉴명 :{menu_name}, 칼로리: {calorie}kcal" 입니다.
# 1일 권장 섭취량 {daily_target} 중 ({daily_target} - {calorie}) 남았습니다.

# 방금 드신 {menu_name}의 열량인 {calorie}kacl를 빼시려면
# xxx(운동명) xx분 (x)kcal,
# yyy(운동명) yy분 (ykcal),
# zzz(운동명) zz분 (z)kcal 을 하셔야합니다.

# 남은 섭취량 ({daily_target} - {calorie})의 추천 식단은
# oooo(음식명) oo(양) o(kcal),
# pppp(음식명) pp(양) p(kcal),
# qqqq(음식명) qq(양) q(kcal)입니다."

# """)


# chain = prompt | llm | output_parser

# # Placeholder values - replace with actual values from your workflow
# placeholders = {
#     'rag_context': rag_context,  # Assuming rag_context is defined in a previous cell
#     'text': text,
#     'threshold': 0.4,  # Example threshold value
#     'menu_name': '해물칼국수',  # Example menu name
#     'calorie': '184.6',  # Example calorie value
#     'daily_target': '2000',  # Example daily target calorie value
# }

# print(chain.invoke(placeholders))

In [5]:
import os
from typing import List, Tuple
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
import openai  # LLM 호출용
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# --- 1. Pinecone & Index 초기화 ---
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
INDEX_NAME = "food-index"
EMBED_MODEL = "text-embedding-3-small"
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(INDEX_NAME)
embeddings = OpenAIEmbeddings(model=EMBED_MODEL)
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

# --- 2. 검색 및 Context 구성 함수 ---
def search_menu(menu_name: str, k: int = 1) -> List[Tuple]:
    return vector_store.similarity_search_with_score(query=menu_name, k=k)

def build_context(matches: List[Tuple]) -> str:
    lines = []
    for doc, score in matches:
        meta = doc.metadata or {}
        name = meta.get("RCP_NM", "알 수 없는 메뉴")
        kcal = meta.get("INFO_ENG", "칼로리 정보 없음")
        lines.append(f"- 메뉴명: {name}, 칼로리: {kcal} (유사도: {score:.2f})")
    return "\n".join(lines)

def ask_llm_calorie(menu_name: str) -> str:
    prompt = (
        f"다음 음식의 대표적인 1인분 칼로리(kcal) 숫자만 알려주세요 **반드시 숫자만 반환!!**:\n"
        f"‘{menu_name}’"
    )
    resp = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role":"user","content":prompt}],
        temperature=0
    )
    return resp.choices[0].message.content.strip()

def get_menu_context_with_threshold(
    menu_name: str,
    k: int = 1,
    threshold: float = 0.4
) -> str:
    matches = search_menu(menu_name, k)
    if not matches or matches[0][1] < threshold:
        calorie = ask_llm_calorie(menu_name)
        return f"- 메뉴명: {menu_name}, 칼로리: {calorie}"
    return build_context(matches)

# --- 3. LLM Chain 설정 ---
llm = ChatOpenAI(model='gpt-4.1', temperature=0.3)
output_parser = StrOutputParser()
prompt = PromptTemplate.from_template("""
[System Instruction]
- 당신은 영양&운동 전문 PT "G-PT"입니다.
- 음식 이미지 분류 결과로 얻은 menu_name과 텍스트 정보를 바탕으로
  섭취 열량 및 남은 권장 열량에 맞는 식단과 운동을 추천합니다.
- Pinecone RAG에서 유사도 0.4 미만일 경우 LLM에게 fallback합니다.
- 1일 권장 섭취량은 사용자의 입력 text 바탕으로 계산을 해서 사용합니다.

[Input Data]
{rag_context}
{text}

[Output Indicator]
" 드신 메뉴인 '메뉴명 :'{menu_name}, 은 '칼로리:' {calorie}kcal" 입니다.
오늘의 1일 권장 섭취량 (권장 kcal) 중 ((권장 kcal) - {calorie}- (이전 식사의 kcal)) 남았습니다.

방금 드신 {menu_name}의 열량인 {calorie}kcal를 빼시려면
xxx(운동명) xx분 (x)kcal,
yyy(운동명) yy분 (y)kcal,
zzz(운동명) zz분 (z)kcal 을 하셔야합니다.

남은 섭취량 ((권장 kcal) - {calorie} - (이전 식사의 kcal))의 추천 식단은
oooo(음식명) oo(양) o(kcal),
pppp(음식명) pp(양) p(kcal),
qqqq(음식명) qq(양) q(kcal)입니다.
"""
)
chain = prompt | llm | output_parser

# --- 4. 파이프라인---
def analyze_meal(rag_context: str, text: str, menu_name: str, calorie: str) -> str:
    return chain.invoke({
        'rag_context': rag_context,
        'text': text,
        'menu_name': menu_name,
        'calorie': calorie,
    })

if __name__ == '__main__':
    # 예시 호출
    rag_ctx = get_menu_context_with_threshold('떡볶이')
    text = "29세, 남성, 180cm, 82kg, 오늘 점심식사고 앞서 계란볶음밥을 아침으로 먹었습니다."
    calorie = ask_llm_calorie('떡볶이')
    result = analyze_meal(rag_ctx, text, '떡볶이', calorie)
    print(result)


드신 메뉴인 '떡볶이'는 '칼로리: 383kcal' 입니다.  
오늘의 1일 권장 섭취량(약 2,400kcal) 중 (2,400 - 383 - 450) = 1,567kcal 남았습니다.  
(아침 계란볶음밥은 평균 450kcal로 계산하였습니다.)

방금 드신 떡볶이의 열량인 383kcal를 빼시려면  
- 빠르게 걷기 60분 (약 380kcal)  
- 자전거 타기(중간 강도) 45분 (약 380kcal)  
- 줄넘기 35분 (약 380kcal)  
을 하셔야 합니다.

남은 섭취량(1,567kcal)의 추천 식단은  
- 닭가슴살 샐러드 200g 220kcal  
- 고구마 200g 260kcal  
- 현미밥 150g 210kcal  
- 두부구이 100g 110kcal  
- 바나나 1개 90kcal  
- 저지방 우유 1컵 120kcal  
총 약 1,010kcal (저녁/간식 분배),  
나머지는 채소, 과일 등으로 채우시면 좋습니다.

운동과 식단 모두 균형 있게 챙기시길 추천드립니다!
